Created: 2020.07.23

Modified: 2020.08.06

## Get the resolutions of all images in all databases,

In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [8]:
import glob 
import numpy as np
import pandas as pd
import nibabel as nib
from pathlib import Path
from fastai.core import parallel

In [9]:
import sys
sys.path.append('functions')

%aimport functions00

from functions00 import *

In [10]:
get_host_info()

************************************************************************************
Settings:
	 HOST: mmiv-ml-titan
	 PATH_ROOT_DATA: /data-10tb/shared/skull/train-3d-iso
	 PATH_GIT_HUB: /data-10tb/marek/github_codes/skull-stripping-1/fastai
************************************************************************************


In [11]:
save_folder = PATH_GIT_HUB / '2.1_resample_jobs'

In [12]:
pth = PATH_ROOT_DATA / '*/*.nii.gz'
files = glob.glob(str(pth))
len(files)

53141

### Get info about all 3D iso images in the train-3d-iso folder

Get image sizes abot isotrobpic images in all databased. This information maight be helpful in decison about image while training NN.

#### Function to run in parallel

In [29]:
def get_shapes(f, index=0):
    return (f, nib.load(f).get_fdata().shape)

In [5]:
# function runs fro all images in all databases!!!
# in f1 we have a list of tuples (image_nama, shape)
run = 0
if run:
    %%time
    f1 = parallel(get_shapes, files, max_workers=64)
    print(len(f1))

In [6]:
# to create a df we can take a calculated result or loaded from a csv file
if 'f1' in locals():
    names, shapes = zip(*f1)
    df = pd.DataFrame.from_dict({'file':names, 'shapes':shapes})
    print('df from calculation')
else:
    df = pd.read_csv(save_folder / 'all_iso_images_shapes.csv')
    print('df loaded from a file')

df loaded from a file


In [7]:
# we can save calculateed df for all images to a csv file
save = 0
if save:
    save_name = f'all_iso_images_shapes.csv'    
    pth = save_folder/save_name

    df.to_csv(pth, index=False)

### Get size info about images in each database

In [8]:
def df_info(df, database):
    un = df.shapes.unique()
    vc = df.shapes.value_counts()
    nu = df.shapes.nunique()
    cn = df.shapes.count()
    print(f'Shape info about: {database} 3D isotropic images\n')
    print(f'*** Total number of {database} images: {cn} ***\n')
    print(f'*** Nuber of unique shape values: {len(un)} ***\n')
    print(f'*** Unique shape and its counts: \n{vc} ***')
    #print(f'\t*** Non unique values: {nu}\n')

In [9]:
def enumerate_db_shapes(df, database, save=False):
    """
    enumrates shapes of images in db and save to csv file in needed.
    
    shapes_id = 1-N, index of possible image shapes in db
    shapes_cnt = number of images wiht this shape within db
    """
    df1 = df.copy()
    vc = df1.shapes.value_counts()
    for k, sh_all in enumerate(zip(vc, vc.index)):
        shnr, sh = sh_all
        #print(k, shnr, sh)
        df1.loc[df1['shapes']==sh, 'shapes_id'] = k+1
        df1.loc[df1['shapes']==sh, 'shapes_cnt'] = shnr
        
        df2 = df1.sort_values(by=['shapes_id'], inplace=False, ascending=False)

    # set max dimension in the 'max_dims' column
    r,c = df2.shape
    mx = tuple(np.array([eval(t) for t in df2.shapes.values]).max(axis=0))
    df2.insert(c, 'max_dims', [mx]*r, False)
    #df1['max_dims'] = [mx]*df1.shape[0]
    
    if save:
        save_name = f'{database}_iso_images_shapes.csv'    
        pth = save_folder/save_name
        df2.to_csv(pth, index=False)    
    return max(mx)

In [10]:
max_dim_for_all = []

#### ADNI

In [11]:
%% time
database = 'ADNI'

df_out = df[df['file'].str.contains(database)]
m = enumerate_db_shapes(df_out, database, save=True)
max_dim_for_all.append(m)
df_info(df_out, database)

Shape info about: ADNI 3D isotropic images

*** Total number of ADNI images: 25935 ***

*** Nuber of unique shape values: 162 ***

*** Unique shape and its counts: 
(211, 240, 170)    6381
(192, 240, 170)    3445
(235, 260, 170)    3015
(204, 256, 170)    2256
(200, 240, 170)     699
                   ... 
(191, 239, 170)       3
(154, 240, 170)       3
(174, 240, 169)       3
(211, 240, 171)       3
(181, 239, 170)       3
Name: shapes, Length: 162, dtype: int64 ***


#### AIBL

In [12]:
%%time
database = 'AIBL'

df_out = df[df['file'].str.contains(database)]
m = enumerate_db_shapes(df_out, database, save=True)
max_dim_for_all.append(m)
df_info(df_out, database)

Shape info about: AIBL 3D isotropic images

*** Total number of AIBL images: 2173 ***

*** Nuber of unique shape values: 1 ***

*** Unique shape and its counts: 
(192, 240, 170)    2173
Name: shapes, dtype: int64 ***


#### IXI

In [13]:
%%time
database = 'IXI'

df_out = df[df['file'].str.contains(database)]
m = enumerate_db_shapes(df_out, database, save=True)
max_dim_for_all.append(m)
df_info(df_out, database)

Shape info about: IXI 3D isotropic images

*** Total number of IXI images: 1743 ***

*** Nuber of unique shape values: 5 ***

*** Unique shape and its counts: 
(180, 240, 170)    1494
(175, 240, 170)     222
(180, 250, 170)      15
(156, 240, 170)       6
(168, 240, 170)       6
Name: shapes, dtype: int64 ***


#### PPMI

In [14]:
%%time
database = 'PPMI'

df_out = df[df['file'].str.contains(database)]
m = enumerate_db_shapes(df_out, database, save=True)
max_dim_for_all.append(m)
df_info(df_out, database)

Shape info about: PPMI 3D isotropic images

*** Total number of PPMI images: 2250 ***

*** Nuber of unique shape values: 11 ***

*** Unique shape and its counts: 
(176, 240, 170)    1794
(176, 256, 170)     279
(192, 240, 170)      69
(176, 250, 170)      51
(256, 240, 170)      12
(160, 240, 170)      12
(175, 240, 170)      12
(211, 253, 170)       9
(192, 256, 170)       6
(144, 240, 170)       3
(176, 248, 170)       3
Name: shapes, dtype: int64 ***


#### SALD

In [15]:
%%time
database = 'SALD'

df_out = df[df['file'].str.contains(database)]
m = enumerate_db_shapes(df_out, database, save=True)
max_dim_for_all.append(m)
df_info(df_out, database)

Shape info about: SALD 3D isotropic images

*** Total number of SALD images: 1472 ***

*** Nuber of unique shape values: 1 ***

*** Unique shape and its counts: 
(176, 256, 170)    1472
Name: shapes, dtype: int64 ***


#### SLIM

In [16]:
%%time
database = 'SLIM'

m = enumerate_db_shapes(df_out, database, save=True)
max_dim_for_all.append(m)
df_info(df_out, database)

Shape info about: SLIM 3D isotropic images

*** Total number of SLIM images: 1472 ***

*** Nuber of unique shape values: 1 ***

*** Unique shape and its counts: 
(176, 256, 170)    1472
Name: shapes, dtype: int64 ***


### The maximal dimensions

In [17]:
max_dim_for_all

[282, 240, 250, 256, 256, 256]

In [19]:
max(max_dim_for_all)

282